# Задача

# Введение
В Контуре мы много работаем с документами: арбитражные иски, госзакупки, исполнительные производства. В данном задании мы предлагаем вам сделать модель, которая поможет отделу госзакупок извлекать 
нужный кусок текста из документа для того, чтобы сформировать анкету заявки. То, какой именно фрагмент текста нужно извлечь, зависит от пункта анкеты, соответствующего документу.
Всего в каждом документе, с которыми вы будет работать, есть 1 из 2-х пунктов анкеты, по которым необходимо извлекать кусочки из текста:
- обеспечение исполнения контракта
- обеспечение гарантийных обязательств

Соответственно, ваша модель, принимая на вход `текст документа` и `наименование одного из двух пунктов`, должна возвращать `соответствующий кусочек текста из текста документа`.

# Данные

### train.json 
Данные для обучения в формате json имеют следующие поля:
- `id`: int - id документа
-  `text`: str - текст документа, в котором может содержаться фрагмент текста, соответствующий пункту анкеты из поля `label`
- `label`: str - название пункта анкеты. Может принимать одно из двух значений: `обеспечение исполнения контракта` или `обеспечение гарантийных обязательств`
- `extracted_part`: dict следующего формата:
    ```
    {
        'text': [фрагмент текста из поля `text`, соответствующий пункту анкеты], 
        'answer_start': [индекс символа начала фрагмента текста в тексте документа],
        'answer_end': [индекс символа конца фрагмента текста в тексте документа]
    }
   ```
  
### test.json

Для демонстрации работы модели используйте данные из файла `test.json`. В нем есть все те же поля, что и в файле `train.json`, кроме поля `extracted_part` - именно его вам и нужно будет добавить,
для того, чтобы мы смогли оценить качество вашей модели.

# Тестовое задание

Для выполнения тестового задания требуется разработать модель, которая будет способна по паре `текст документа` и `пункт анкеты` извлекать из текста документа нужный фрагмент текста. 
Обучив модель, добавьте в файл `test.json` поле `extracted_part` в том же формате, что и в файле `train.json`. Новый файл назовите `predictions.json`

**Подсказка**: изучив данные, вы можете заметить, что у части наблюдений отсутствует фрагмент текста к извлечению (пустая строка внутри поля `extracted_part` с `answer_start` и
`answer_end` равными нулю). Это означает, что в тексте документа нет нужного фрагмента текста, соответствующего пункту анкеты. Учтите это в обучении вашей модели и при формировании
файла с ответами.

# Критерии оценки
1. Для оценки финального решения будет использоваться метрика `Accuracy`: доля наблюдений, в которых извлеченный моделью фрагмент текста полностью соответствует фактически
   требуемому фрагменту.
2. Чистота кода, оформление и понятность исследования.

# Требования к решению
В качестве решения мы ожидаем zip-архив со всеми *.py и *.ipynb файлами в папке solution и файлом `predictions.json` в корне. Формат имени zip-архива: LastName_FirstName.zip (пример Ivanov_Ivan.zip).
Файл `predictions.json` должен включать в себя колонки `id`, `text`, `label`, содержащие те же данные, что и исходный файл `test.json`, а также колонку `extracted_part` в том же
формате, что и в файле `train.json`
Разметка тестового набора данных и включение его в обучение/валидацию запрещены.

В папке solution должно быть отражено исследование и весь код, необходимый для воспроизведения исследования.

Успехов!

## Генерация датасетов

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import json
import numpy as np
import nltk
import re
import matplotlib.pyplot as plt
import seaborn as sns
import os 
from tensorflow import keras
import tensorflow as tf 
import tensorflow as tf 
from keras import models
from keras import layers
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import spacy
import random
from spacy.lang.ru import Russian
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


2023-04-11 16:50:28.789627: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-11 16:50:28.789653: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
train_df = pd.read_json('dataset/train.json', orient='records')

In [3]:
train_df

,id,text,label,extracted_part
0,809436509,Извещение о проведении открытого конкурса в эл...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
1,854885310,ТРЕБОВАНИЯ К СОДЕРЖАНИЮ ЗАЯВКИ участника запро...,обеспечение исполнения контракта,{'text': ['Поставщик должен предоставить обесп...
2,4382157,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
3,184555082,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
4,211645258,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
...,...,...,...,...
1794,938335285,"«УТВЕРЖДАЮ» Директор МУП ""ПКП и ОРТ"" _________...",обеспечение гарантийных обязательств,{'text': ['Заказчиком установлено требование о...
1795,214609521,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯ...,обеспечение гарантийных обязательств,"{'text': [''], 'answer_start': [0], 'answer_en..."
1796,94811748,ДОГОВОР ПОДРЯДА № _____ «___»___________20____...,обеспечение гарантийных обязательств,"{'text': [''], 'answer_start': [0], 'answer_en..."
1797,903507299,ДОКУМЕНТАЦИЯ О ЗАКУПКЕ ЗАПРОС ПРЕДЛОЖЕНИЙ В ЭЛ...,обеспечение гарантийных обязательств,"{'text': [''], 'answer_start': [0], 'answer_en..."


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1799 non-null   int64 
 1   text            1799 non-null   object
 2   label           1799 non-null   object
 3   extracted_part  1799 non-null   object
dtypes: int64(1), object(3)
memory usage: 56.3+ KB


In [5]:

# Сброс ограничений на количество выводимых рядов
#pd.set_option('display.max_rows', None)
 
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
 
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

In [6]:
print(train_df['extracted_part'].head(1).astype( str ))

0    {'text': ['Размер обеспечения исполнения контракта 6593.25 Российский рубль'], 'answer_start': [1279], 'answer_end': [1343]}
Name: extracted_part, dtype: object


In [7]:
obj = dict(train_df['extracted_part'].values[0])
print(obj['text'], obj['answer_start'],obj['answer_end'])

['Размер обеспечения исполнения контракта 6593.25 Российский рубль'] [1279] [1343]


In [8]:
print(train_df['text'].values[0])

Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета в соответствии с п.16 ч. 1 

# Обучающий датасет формирование

In [9]:
train_df['label'].value_counts()

обеспечение исполнения контракта        988
обеспечение гарантийных обязательств    811
Name: label, dtype: int64

In [10]:
train_df['text_str'] = train_df.apply(lambda row: ''.join(dict(row['extracted_part'])['text']), axis=1)
train_df['text_start'] = train_df.apply(lambda row: dict(row['extracted_part'])['answer_start'][0], axis=1)
train_df['text_end'] = train_df.apply(lambda row: dict(row['extracted_part'])['answer_end'][0], axis=1)

In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1799 non-null   int64 
 1   text            1799 non-null   object
 2   label           1799 non-null   object
 3   extracted_part  1799 non-null   object
 4   text_str        1799 non-null   object
 5   text_start      1799 non-null   int64 
 6   text_end        1799 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 98.5+ KB


In [12]:
train_df

,id,text,label,extracted_part,text_str,text_start,text_end
0,809436509,"Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета в соответствии с п.16 ч. 1 ст. 42 Закона № 44-ФЗ ""Номер расчётного счёта"" 00000000000000000000 ""Номер лицевого счёта"" См. прилагаемые документы ""БИК"" 000000000 ""Наименование кредитной организации"" ""Номер корреспондентского счета"" Обеспечение исполнения контракта Требуется обеспечение исполнения контракта Размер обеспечения исполнения контракта 6593.25 Российский рубль Порядок обеспечения исполнения контракта, требования к обеспечению Исполнение контракта, гарантийные обязательства могут обеспечиваться предоставлением независимой гарантии, соответствующей требованиям ст. 45 Федерального закона № 44-ФЗ, или внесением денежных средств на указанный заказчиком счет, на котором в соответствии с законодательством Российской Федерации учитываются операции со средствами, поступающими заказчику. Способ обеспечения исполнения контракта, гарантийных обязательств, срок действия независимой гарантии определяются в соответствии с требованиями Федерального закона № 44-ФЗ участником закупки, с которым заключается контракт, самостоятельно. При этом срок действия независимой гарантии должен превышать предусмотренный контрактом срок исполнения обязательств, которые должны быть обеспечены такой независимой гарантией, не менее чем на один месяц, в том числе в случае его изменения в соответствии со ст. 95 Федерального закона № 44-ФЗ. Порядок предоставления и требования о Информация и документы, подтверждающие соответствие участников закупки дополнительным требованиям: 1) исполненный договор; 2) акт приемки оказанных услуг и (или) поставленных товаров, подтверждающий цену оказанных услуг и (или) поставленных товаров 4",обеспечение исполнения контракта,"{'text': ['Размер обеспечения исполнения контракта 6593.25 Российский рубль'], 'answer_start': [1279], 'answer_end': [1343]}",Размер обеспечения исполнения контракта 6593.25 Российский рубль,1279,1343
1,854885310,"ТРЕБОВАНИЯ К СОДЕРЖАНИЮ ЗАЯВКИ участника запроса котировок в электронной форме Заявка на участие в запросе котировок в электронной форме должна содержать следующие информацию и документы: 1) об участнике запроса котировок носят на банковский счет, открытый в банке, включенном в перечень, который утвержден распоряжением Правительства Российской Федерации от 13.07.2018 N 1451-р. Требования к банкам, договору специального счета, к порядку использования имеющегося у участника закупки банковского счета в качестве специального счета Правительство РФ установило в постановлении от 30.05.2018 №626. Независимая гарантия должна соответствовать требованиям статьи 45 Закона № 44-ФЗ Реквизиты счета, на котором в соответствии с законодательством Российской Федерации учитываются операции со средствами, поступающими заказчику: Наименование заказчика: Санкт-Петербургское государственное бюджетное учреждение здравоохранения «Городская поликлиника № 95» (СПб ГБУЗ ГП № 95) Реквизиты: ИНН 7817026947, КПП 781701001 Реквизиты банковского счета: Северо-Западное ГУ БАНКА РОССИИ//УФК 

In [13]:
train_df['target']=1

In [14]:
train_df['ent']=''
train_df['pre']=0

# Сформируем валидационную и обучающую выборки.

In [15]:
X = train_df
y = train_df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [16]:
X_train.shape

(1529, 10)

In [17]:
y_train.shape

(1529, 10)

In [18]:
X_test.shape

(270, 10)

In [19]:
y_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270 entries, 1022 to 938
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              270 non-null    int64 
 1   text            270 non-null    object
 2   label           270 non-null    object
 3   extracted_part  270 non-null    object
 4   text_str        270 non-null    object
 5   text_start      270 non-null    int64 
 6   text_end        270 non-null    int64 
 7   target          270 non-null    int64 
 8   ent             270 non-null    object
 9   pre             270 non-null    int64 
dtypes: int64(5), object(5)
memory usage: 23.2+ KB


In [20]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1799 non-null   int64 
 1   text            1799 non-null   object
 2   label           1799 non-null   object
 3   extracted_part  1799 non-null   object
 4   text_str        1799 non-null   object
 5   text_start      1799 non-null   int64 
 6   text_end        1799 non-null   int64 
 7   target          1799 non-null   int64 
 8   ent             1799 non-null   object
 9   pre             1799 non-null   int64 
dtypes: int64(5), object(5)
memory usage: 140.7+ KB


# nlp 

# Сформируем TRAIN_DATA

In [21]:
TRAINING_DATA=[]
for index, row in X_train.iterrows():
    TRAINING_DATA.append((row['text'],{"entities":[(row['text_start'],row['text_end'],row['label'])]}))

In [22]:
TRAINING_DATA[0]

('УТВЕРЖДАЮ Директор по комплектации и закупкам АО АСЭ В.Ю. Синев ______________________ подпись «____» ______________ 2022 г. УТВЕРЖДАЮ Президент АО ИК «АСЭ» В.И. Лимаренко ______________________ подпись «____» марта 2018 г. УТВЕРЖДАЮ  пунктом 14.3 Части 2 закупочной документации), указанный в анкете участника закупки (Форма 2 подраздела 5.1). Лицо, с которым заключается договор, обязано предоставить заказчику подписанный и заверенный печатью со своей стороны оригинал договора (в двух экземплярах) в течение 4 рабочих дней со дня направления указанного проекта договора заказчиком на адрес электронной почты, указанный в анкете участника закупки (Форма 2 подраздела 5.1). В случае предоставления приоритета согласно Постановлению 925 при уклонении победителя закупки от заключения договора, договор заключается с участником закупки, занявшим следующее место в ранжировке. 20) Обеспечение исполнения обязательств по договору: форма, размер и срок предоставления обеспечения исполнения обязательс

# Обучим  модель.

nlp = spacy.blank("ru")
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner)
ner.add_label("обеспечение исполнения контракта")
ner.add_label("обеспечение гарантийных обязательств")
nlp.begin_training()

nlp=spacy.load("model99")

for i in range(100,150):
    random.shuffle(TRAINING_DATA)
    for batch in spacy.util.minibatch(TRAINING_DATA):
        texts = [text for text, annotation in batch]
        annotations = [annotation for text, annotation in batch]
        nlp.update(texts, annotations)
    
        #Валидация 
    for index, row in X_train.iterrows():
        doc = nlp(row['text'])
        if doc.ents:
            X_train['ent'].loc[index]=str(doc.ents[0])
        else:
            X_train['ent'].loc[index]=''
        
        
    for index, row in X_train.iterrows():
        if X_train['ent'].loc[index]==X_train['text_str'].loc[index]:
            X_train['pre'].loc[index]=1
        
    print('Эпоха', i, )
    print('accuracy обучающей выборки', accuracy_score(X_train['target'], X_train['pre']))
    print('-----------------------------------------------------------------------------')
        
        
        
        
    for index, row in X_test.iterrows():
        doc = nlp(row['text'])
        if doc.ents:
            #print(doc.ents[0])
            X_test['ent'].loc[index]=str(doc.ents[0])
        else:
            X_test['ent'].loc[index]=''
        
        
    for index, row in X_test.iterrows():
        if X_test['ent'].loc[index]==X_test['text_str'].loc[index]:
            X_test['pre'].loc[index]=1
        
    print('Эпоха', i, )
    print('accuracy вылидационной  выборки', accuracy_score(X_test['target'], X_test['pre']))
    print('-----------------------------------------------------------------------------')
        #Валидация
        
    nlp.to_disk("model"+str(i))

In [23]:
nlp=spacy.load("model149")

In [55]:

doc = nlp(
'''УТВЕРЖДЕНО решением Единой закупочной комиссии ООО «Технический заказчик Фонда развития территорий» Извещение о проведении открытого конкурса в электронной форме и документация о проведении открытого конкурса в электронной форме на выполнение ия участникам закупки документации о закупке – с 02.09.2022 по 21.09.2022 13.6 В случае если запрос на предоставление документации о закупке поступил на почтовый адрес, указанный в пункте 2 настоящего извещения, заказчик, организатор закупки предоставляет документацию о закупке по адресу, указанному в пункте 2 настоящего извещения. 13.7 В случае если запрос на предоставление документации о закупке поступил на адрес электронной почты, указанный в пункте 2 настоящего извещения, заказчик, организатор закупки направляет документацию о закупке на адрес электронной почты, указанный в запросе на предоставление документации о закупке. 13.8 Документация о закупке заинтересованному лицу предоставляется бесплатно. 14. Информация о размере обеспечения заявок на участие в закупке 14.1 Обеспечение заявки – не устанавливается 14.2 Информация о порядке и способе обеспечения заявок на участие в закупке указана в п. 4 документации о закупке. 15. Информация о размере обеспечения исполнения договора 15.1 Обеспечение исполнения договора устанавливается в размере 2% - 43 375 595 (сорок три миллиона триста семьдесят пять тысяч пятьсот девяносто пять) рублей, 65 копеек. 15.2 Информация о порядке и способе обеспечения исполнения договора на участие в закупке указана в п. 5 документации о закупке. 15.3 Размер обеспечения гарантийных обязательств установлен в размере 2% (два процента) от цены договора. 15.4 Порядок и способ предоставления обеспечения гарантийных обязательств указан в п.8.4.1. Проекта договора. 16. Антидемпинговые меры 16.1. Под демпингом понимается предложение цены, сделанное участником закупки, на 25 (двадцать пять) процентов и более ниже начальной (максимальной) цены договора (цены единицы услуги) (далее – Предложение с признаками демпинга), за исключением случаев предоставления участником закупки в составе заявки информации о наличии у него опыта поставки товаров, выполнения работ, оказания услуг по предмету закупки на сумму не менее 50 (пятидесяти) процентов от начальной (максимальной) цены договора/предельной цены договора, устан Руководитель/уполномоченный представитель участника закупки или каждого лица, выступающего на стороне коллективного участника закупки ________________ (Фамилия И.О.) (реквизиты документа, подтверждающего полномочия) (подпись)''')

for ent in doc.ents:
    if ent.label_ == "обеспечение исполнения контракта" and ent.start != 0:
        print(ent.text, ent.label_)



Обеспечение исполнения договора устанавливается в размере 2% обеспечение исполнения контракта


In [47]:
ents = [(e.text, e.label_,) for e in doc.ents]
print(ents)  

[('Обеспечение исполнения договора устанавливается в размере 2%', 'обеспечение исполнения контракта')]


In [48]:
from spacy import displacy
displacy.render(doc, style='ent', jupyter=True)

In [49]:
nlp.get_pipe("ner").labels

('обеспечение гарантийных обязательств', 'обеспечение исполнения контракта')

# Валидация на всей обучающей выборке.

In [56]:
for index, row in train_df.iterrows():
    doc = nlp(row['text'])
    if doc.ents:
        if ent.label_ ==train_df['label'].iloc[index] and ent.start != 0:
            print(doc.ents[0])
            train_df['ent'].iloc[index]=str(doc.ents[0])
    else:
        train_df['ent'].iloc[index]=''
    

Размер обеспечения исполнения контракта 6593.25 Российский рубль
Поставщик должен предоставить обеспечение исполнения контракта в размере 10 % от цены Контракта.
Размер обеспечения исполнения контракта 10.00%
Размер обеспечения исполнения контракта 10.00%
Размер обеспечения исполнения контракта 10.00%
Размер обеспечения исполнения контракта устанавливается в размере 5 (пять) процентов от цены, по которой заключается контракт и составляет ___
Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта
Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта
Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта
Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта
Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены

Размер обеспечения исполнения контракта составляет 10 (десять) процентов начальной (максимальной) цены контракта: 8 903 рублей
Размер обеспечения исполнения контракта 5.00%
Размер обеспечения исполнения контракта 9523.60 Российский рубль
Размер обеспечения исполнения контракта 10.00%
Размер обеспечения исполнения контракта 5.00%
Размер обеспечения исполнения контракта 15.00%
Размер обеспечения исполнения контракта 3354.78 Российский рубль
Размер обеспечения исполнения контракта 5.00%
Размер обеспечения исполнения контракта 1383.33 Российский рубль
Размер обеспечения исполнения контракта 5.00%
обеспечения исполнения контракта в размере, превышающем в 1,5 раза размер обеспечения исполнения контракта, указанный в извещении, но не менее чем 10 процентов от начальной (максимальной) цены контракта или от цены
Поставщик при заключении Контракта должен предоставить Заказчику обеспечение исполнения Контракта в размере 10 % ( __________ рублей)
Поставщик при заключении Договора должен предостави

Размер обеспечения исполнения контракта 17389.85 Российский рубль
Размер обеспечения исполнения контракта 120711.45 Российский рубль
Размер обеспечения исполнения контракта 0.50%
Размер обеспечения исполнения контракта 10.00%
Размер обеспечения исполнения государственного контракта составляет 30 % от начальной (максимальной) цены контракта
Размер обеспечения исполнения контракта 10.00%
размер обеспечения исполнения контракта - 5 (пяти) процентов цены контракта.
Поставщик при заключении Контракта должен предоставить Заказчику обеспечение исполнения Контракта в размере 10 % от начальной (максимальной) цены контракта
Размер обеспечения исполнения Контракта составляет ______________рублей, что составляет 5 процентов от начальной (максимальной) цены Контракта.
Размер обеспечения исполнения контракта 8536320.00 Российский рубль
Размер обеспечения исполнения контракта 10.00%
Обеспечение исполнения Контракта устанавливается в размере 5 процентов от Цены Контракта.
Размер обеспечения исполнения

Размер обеспечения исполнения контракта 10.00%
Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта
Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта
Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта
Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта
Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта
Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта
Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта
Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта
Размер обеспечение исполнения контракта устанавли

Размер обеспечения исполнения контракта: 10% от начальной (максимальной) цены Контракта.
Размер обеспечения исполнения контракта 5.00%
Размер обеспечения исполнения контракта 5.00%
Размер обеспечения исполнения контракта 877.50 Российский рубль
Обеспечение исполнения контракта устанавливается в размере 5 % от цены
Размер обеспечения исполнения контракта 2260.52 Российский рубль
Обеспечение исполнения Контракта устанавливается в размере 10% от цены Контракта.
Размер обеспечения исполнения Контракта составляет 10% от начальной (максимальной) цены Контракта.
Размер обеспечения исполнения контракта 7257.10 Российский рубль
Обеспечение исполнения настоящего Контракта установлено в размере __________ рублей _______ копеек (5 процентов начальной (максимальной) цены Контракта).
Размер обеспечения исполнения контракта 5.00%
Размер обеспечения исполнения контракта 10.00%
Размер обеспечения исполнения контракта 5,00 %
Размер обеспечения исполнения контракта 10.00%
Размер обеспечения исполнения ко

...

Установить обеспечение исполнения Договора в размере 5% (пять) процентов от начальной (максимальной) цены Договора
Обеспечение исполнения договора в размере не менее 5% от цены договора
Размер обеспечения исполнения договора, руб. 32 550,00
Поставщик до заключения Договора предоставляет Заказчику обеспечение исполнения Договора в размере 5% Начальной максимальной цены Договора
такой участник обязан предоставить обеспечение исполнения договора в размере 5 процентов от начальной (максимальной) цены договора (цены лота)
Размер обеспечения исполнения Договора устанавливается в размере 5% от НМЦД
Обеспечение исполнения Договора составляет 5 % от начальной (максимальной) цены Договора в сумме _________
договор заключается только после предоставления таким участником обеспечения исполнения договора в размере, превышающем в полтора раза размер обеспечения исполнения договора, указанный в закупочной документации, но не менее чем в размере аванса (если договором предусмотрена выплата аванса). 10

Размер обеспечения договора: не может превышать 5 процентов начальной (максимальной) цены договора
Размер обеспечения исполнения договора 4% начальной (максимальной) цены договора
Размер обеспечения исполнения договора составляет: 5 % от начальной (максимальной) цены договора
Размер обеспечения исполнения договора, срок и порядок его предоставления 5,00 % от начальной (максимальной) цены договора
Обеспечение исполнения обязательств по договору: требуется. - не менее 5 % (пять процентов) от цены договора.
Размер обеспечения исполнения договора: 5 % начальной (максимальной) цены Договора[footnoteRef:1]
Размер обеспечения исполнения договора составляет 100% от цены договора.
Размер обеспечения исполнения договора 3 % от начальной (максимальной) цены договора
не предусмотрено увеличенное от первоначально установленного обеспечения исполнения договора в 1,5 (полтора) раза, но не более 5% (пяти) от начальной (максимальной) цены договора
Размер обеспечения исполнения договора 2 606 259,96 руб

участник обязан предоставить обеспечение исполнения договора в размере 5 процентов от начальной (максимальной) цены договора (цены лота)
В документации о закупке может быть установлено требование к обеспечению исполнения договора в размере не более 5 (пяти) процентов от начальной (максимальной) цены договора
Размер обеспечения исполнения договора: установлен: 1% от НМЦД – 1 201,67 рублей.
Размер обеспечения исполнения договора: 5% от начальной (максимальной) цены договора
не предусмотрено увеличенное от первоначально установленного обеспечения исполнения договора в 1,5 (полтора) раза, но не менее 5% (пяти) от НМЦ
обеспечение исполнения договора в размере 5% от цены договора.
обеспечение исполнения договора в размере 5% от цены договора.
Размер обеспечения исполнения договора установлен в размере 5% (пяти процентов) от начальной (максимальной) цены договора
обеспечение исполнения договора в размере 5% от цены договора в форме безотзывной банковской гарантии
обеспечение исполнения догово

Обеспечение исполнения обязательств по Договору, кроме гарантийных обязательств (обеспечение договора) установлено в размере 5 % от цены Договора, что составляет что составляет ________________ руб. ___коп. (вариант абзаца без антидемпинга).
Размер обеспечения исполнения договора составляет: 50% от начальной (максимальной) цены договора
Обеспечение исполнения Договора: 5% от начальной (максимальной) цены Договора
Размер обеспечения договора 5 % от НМЦ
Заказчик вправе установить в документации о закупке требование к обеспечению исполнения договора в размере не более 5 (пяти) процентов от начальной (максимальной) цены договора
Размер обеспечения исполнения договора составляет 5 % (пять) от начальной (максимальной) цены договора
Размер обеспечения договора 10 % от НМЦ оставляет:228 416,66 руб.
Размер обеспечения исполнения договора 5% от НМЦ составляет: 57 816,88 рублей
Размер обеспечения: 50 % от цены договора на выполнение строительно-монтажных работ
Размер обеспечения исполнения Догово

Размер обеспечения исполнения Договора составляет 5 % от начальной (максимальной) цены Договора
Размер обеспечения исполнения Договора составляет 5 % от начальной (максимальной) цены Договора
Если в извещении о закупке и (или) документации о закупке установлено требование к обеспечению исполнения договора, размер такого обеспечения: 1) не может превышать 5 процентов начальной (максимальной) цены договора (цены лота)
Необходимость предоставления обеспечения исполнения по Договору в размере не более 5% (пяти процентов) от начальной (максимальной) цены договора с учетом НДС
Необходимость предоставления обеспечения исполнения по Договору в размере не более 5% (пяти процентов) от начальной (максимальной) цены договора с учетом НДС
Исполнитель предоставляет Заказчику обеспечение исполнения Договора в размере 5% от начальной (максимальной) цены договора, что составляет _
Размер обеспечения исполнения договора: 0% от НМЦД
Размер обеспечения исполнения договора 5 (пять) процентов от начальной (

In [57]:
for index, row in train_df.iterrows():
        if train_df['ent'].iloc[index]==train_df['text_str'].iloc[index]:
            train_df['pre'].iloc[index]=1
            
    
    

In [58]:

accuracy_score(train_df['target'], train_df['pre'])

0.924958310172318

In [63]:
train_df.query('pre==0')[['text_str','ent']].head(50)

,text_str,ent
45,"Обеспечение исполнения Контракта устанавливается в размере 0,5 % от цены Контракта, что составляет ___________ (____________","Обеспечение исполнения Контракта устанавливается в размере 0,5 % от цены Контракта, что составляет ___________ (____________)"
67,"Размер обеспечения исполнения Контракта составляет 5% (пять процентов) от цены Контракта, что составляет ___________ ( ) рублей","Размер обеспечения исполнения Контракта составляет 5% (пять процентов) от цены Контракта, что составляет ___________ ( ) рублей копеек."
74,"Поставщик при заключении Контракта должен предоставить Заказчику обеспечение исполнения Контракта в размере 10 % начальной (максимальной) цены контракта – 10 080,00 руб.",10.1.Поставщик при заключении Контракта должен предоставить Заказчику обеспечение исполнения Контракта в размере 10 % начальной (максимальной) цены контракта
87,"Поставщик внес обеспечение исполнения Контракта в размере __ , что составляет 5 %","Поставщик внес обеспечение исполнения Контракта в размере __ , что составляет 5 % Цены Контракта в форме безотзывной независимой гарантии"
92,"Обеспечение исполнения Контракта составляет 10% от цены Контракта(цены лота),что составляет_________ (_____","Обеспечение исполнения Контракта составляет 10% от цены Контракта(цены лота),что составляет_________ (_________________________)"
108,"таким участником обеспечения исполнения контракта в размере, превышающем в 1,5 раза размер обеспечения исполнения контракта, указанный в извещении, но не менее чем 10 процентов от начальной (максимальной) цены контракта","обеспечения исполнения контракта в размере, превышающем в 1,5 раза размер обеспечения исполнения контракта, указанный в извещении, но не менее чем 10 процентов от начальной (максимальной) цены контракта или от цены"
109,Поставщик при заключении Контракта должен предоставить Заказчику обеспечение исполнения Контракта в размере 10 % ( __________ рублей) начальной (максимальной) цены контракта.,Поставщик при заключении Контракта должен предоставить Заказчику обеспечение исполнения Контракта в размере 10 % ( __________ рублей)
132,Обеспечение исполнения настоящего Контракта установлено в размере 5% от цены контракта и составляет _______ (___________________,Обеспечение исполнения настоящего Контракта установлено в размере 5% от цены контракта и составляет _______ (________________________________________
162,Размер обеспечения исполнения Контракта составляет 5 процентов цены Контракта в размере __________ (________) рублей ___,Размер обеспечения исполнения Контракта составляет 5 процентов цены Контракта в размере __________ (________) рублей ___ копеек.[footnoteRef:4]
173,"Размер обеспечения исполнения Контракта устанавливается в размере 10 % от цены, по которой заключается Контракт и составляет______","Размер обеспечения исполнения Контракта устанавливается в размере 10 % от цены, по которой заключается Контракт и составляет____________ руб. ___ коп."
